In [21]:
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.express as px
from Feature_Engineering import generate_ema
import pandas as pd

In [40]:
#categorical_ema, sliding_ema = generate_ema()
sentiment_df = pd.read_csv('Data/articles_features.csv')
sentiment_df

,Title,Time,Link,Subdomain,Sentiment
0,"CVC Set to Launch IPO, Seeks Valuation of Up t...",2024-04-14 12:02,https://www.wsj.com/business/energy-oil/cvc-se...,Business,2
1,"Ready, Set, Fix That Plane in 15 Minutes or Less!",2024-04-14 10:02,https://www.wsj.com/business/airlines/airplane...,Business,1
2,The Home-Solar Boom Gets a ‘Gut Punch’,2024-04-14 09:02,https://www.wsj.com/business/energy-oil/the-ho...,Business,0
3,A Day in the Life of a Walmart Manager Who Mak...,2024-04-14 07:02,https://www.wsj.com/business/retail/a-day-in-t...,Business,2
4,$850 Millennium Falcons and $680 Titanics: Gro...,2024-04-13 00:00,https://www.wsj.com/business/retail/850-millen...,Business,2
...,...,...,...,...,...
310,This $77 Million Aspen Ranch is Colorado’s Mos...,2024-04-11 17:08,https://www.wsj.com/real-estate/luxury-homes/t...,Real-Estate,2
311,Late Designer Max Azria’s Massive Compound Is ...,2024-04-11 13:08,https://www.wsj.com/real-estate/luxury-homes/l...,Real-Estate,2
312,The Party’s Over for Coachella’s Glitziest Ren...,2024-04-11 00:00,https://www.wsj.com/real-estate/luxury-homes/t...,Real-Estate,0
313,Movie Theaters Have Weird Real Estate. It’s Sa...,2024-04-08 00:00,https://www.wsj.com/real-estate/commercial/mov...,Real-Estate,2


In [41]:
sentiment_df['Time'] = pd.to_datetime(sentiment_df['Time'])
sentiment_df['Time'] = sentiment_df['Time'].dt.date
sentiment_df

,Title,Time,Link,Subdomain,Sentiment
0,"CVC Set to Launch IPO, Seeks Valuation of Up t...",2024-04-14,https://www.wsj.com/business/energy-oil/cvc-se...,Business,2
1,"Ready, Set, Fix That Plane in 15 Minutes or Less!",2024-04-14,https://www.wsj.com/business/airlines/airplane...,Business,1
2,The Home-Solar Boom Gets a ‘Gut Punch’,2024-04-14,https://www.wsj.com/business/energy-oil/the-ho...,Business,0
3,A Day in the Life of a Walmart Manager Who Mak...,2024-04-14,https://www.wsj.com/business/retail/a-day-in-t...,Business,2
4,$850 Millennium Falcons and $680 Titanics: Gro...,2024-04-13,https://www.wsj.com/business/retail/850-millen...,Business,2
...,...,...,...,...,...
310,This $77 Million Aspen Ranch is Colorado’s Mos...,2024-04-11,https://www.wsj.com/real-estate/luxury-homes/t...,Real-Estate,2
311,Late Designer Max Azria’s Massive Compound Is ...,2024-04-11,https://www.wsj.com/real-estate/luxury-homes/l...,Real-Estate,2
312,The Party’s Over for Coachella’s Glitziest Ren...,2024-04-11,https://www.wsj.com/real-estate/luxury-homes/t...,Real-Estate,0
313,Movie Theaters Have Weird Real Estate. It’s Sa...,2024-04-08,https://www.wsj.com/real-estate/commercial/mov...,Real-Estate,2


In [42]:
new_df = sentiment_df.groupby(['Subdomain', 'Time'])['Sentiment'].mean().reset_index()
new_df['Time'] = pd.to_datetime(new_df['Time'])
new_df['Time'] = new_df['Time'].dt.date
new_df

,Subdomain,Time,Sentiment
0,Articles,2024-03-09,2.000000
1,Articles,2024-03-19,0.666667
2,Articles,2024-03-24,0.750000
3,Articles,2024-03-25,0.000000
4,Articles,2024-03-26,0.000000
...,...,...,...
97,Real-Estate,2024-04-10,1.600000
98,Real-Estate,2024-04-11,1.333333
99,Real-Estate,2024-04-12,1.000000
100,Real-Estate,2024-04-13,0.000000


In [43]:
date_range_df = pd.DataFrame()
for subdomain, group in new_df.groupby('Subdomain'):
    min_date = group['Time'].min()
    max_date = group['Time'].max()
    all_dates = pd.date_range(start=min_date, end=max_date)
    subdomain_dates_df = pd.DataFrame({'Time': all_dates, 'Subdomain': subdomain})
    date_range_df = pd.concat([date_range_df, subdomain_dates_df]).reset_index(drop=True)

# Convert the 'Time' column in date_range_df to datetime
date_range_df['Time'] = pd.to_datetime(date_range_df['Time'])
date_range_df['Time'] = date_range_df['Time'].dt.date

# Merge the calculated sentiment data with the DataFrame containing all possible dates
merged_df = pd.merge(date_range_df, new_df, on=['Subdomain', 'Time'], how='left')
merged_df

,Time,Subdomain,Sentiment
0,2024-03-09,Articles,2.000000
1,2024-03-10,Articles,NaN
2,2024-03-11,Articles,NaN
3,2024-03-12,Articles,NaN
4,2024-03-13,Articles,NaN
...,...,...,...
197,2024-04-10,Real-Estate,1.600000
198,2024-04-11,Real-Estate,1.333333
199,2024-04-12,Real-Estate,1.000000
200,2024-04-13,Real-Estate,0.000000


In [44]:
merged_df['Sentiment'].fillna(2, inplace=True)
merged_df['Sentiment'] = (merged_df['Sentiment'] * 25).round(0).astype(int)
merged_df

C:\Users\sasre\AppData\Local\Temp\ipykernel_37484\1391422873.py:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,Time,Subdomain,Sentiment
0,2024-03-09,Articles,50
1,2024-03-10,Articles,50
2,2024-03-11,Articles,50
3,2024-03-12,Articles,50
4,2024-03-13,Articles,50
...,...,...,...
197,2024-04-10,Real-Estate,40
198,2024-04-11,Real-Estate,33
199,2024-04-12,Real-Estate,25
200,2024-04-13,Real-Estate,0


In [46]:
bucket_sizes = [4, 12, 30]

ema_data = pd.DataFrame()
for subdomain, group in merged_df.groupby('Subdomain'):
    subdomain_ema = pd.DataFrame({'Time': group['Time'], 'Subdomain': subdomain})
    for bucket_size in bucket_sizes:
        ema_column_name = f'EMA_{bucket_size}_days'
        ema_values = group['Sentiment'].ewm(span=bucket_size, adjust=False).mean()
        subdomain_ema[ema_column_name] = (ema_values).astype(int)
    ema_data = pd.concat([ema_data, subdomain_ema])

# Reset index
ema_data.reset_index(drop=True, inplace=True)
ema_data

,Time,Subdomain,EMA_4_days,EMA_12_days,EMA_30_days
0,2024-03-09,Articles,50,50,50
1,2024-03-10,Articles,50,50,50
2,2024-03-11,Articles,50,50,50
3,2024-03-12,Articles,50,50,50
4,2024-03-13,Articles,50,50,50
...,...,...,...,...,...
197,2024-04-10,Real-Estate,34,36,36
198,2024-04-11,Real-Estate,34,35,36
199,2024-04-12,Real-Estate,30,34,35
200,2024-04-13,Real-Estate,18,28,33


In [47]:
selected_subdomain = 'Business'  # Replace 'YourSelectedSubdomain' with the desired subdomain
selected_window_size = 4  # Replace 4 with the desired EMA window size

# Filter data for the selected subdomain and window size
filtered_data = ema_data[(ema_data['Subdomain'] == selected_subdomain) & (ema_data[f'EMA_{selected_window_size}_days'].notna())]

# Create a trace for the selected data
trace = go.Scatter(
    x=filtered_data['Time'],
    y=filtered_data[f'EMA_{selected_window_size}_days'],
    mode='lines',
    name=f'{selected_subdomain} - EMA {selected_window_size} days'
)

In [48]:
layout = go.Layout(
    title='EMA Line Graph',
    xaxis=dict(title='Time'),
    yaxis=dict(title='EMA Value', range=[0, 100])
)

# Create figure
fig = go.Figure(data=[trace], layout=layout)

# Show figure
fig.show()

In [51]:
def calculate_sliding_bucket_ema(sentiment_df):
    sentiment_df['Time'] = pd.to_datetime(sentiment_df['Time'])
    sentiment_df['Time'] = sentiment_df['Time'].dt.date
    new_df = sentiment_df.groupby(['Subdomain', 'Time'])['Sentiment'].mean().reset_index()
    new_df['Time'] = pd.to_datetime(new_df['Time'])
    new_df['Time'] = new_df['Time'].dt.date

    date_range_df = pd.DataFrame()
    for subdomain, group in new_df.groupby('Subdomain'):
        min_date = group['Time'].min()
        max_date = group['Time'].max()
        all_dates = pd.date_range(start=min_date, end=max_date)
        subdomain_dates_df = pd.DataFrame({'Time': all_dates, 'Subdomain': subdomain})
        date_range_df = pd.concat([date_range_df, subdomain_dates_df]).reset_index(drop=True)

    # Convert the 'Time' column in date_range_df to datetime
    date_range_df['Time'] = pd.to_datetime(date_range_df['Time'])
    date_range_df['Time'] = date_range_df['Time'].dt.date

    # Merge the calculated sentiment data with the DataFrame containing all possible dates
    merged_df = pd.merge(date_range_df, new_df, on=['Subdomain', 'Time'], how='left')

    merged_df['Sentiment'].fillna(2, inplace=True)
    merged_df['Sentiment'] = (merged_df['Sentiment'] * 25).round(0).astype(int)

    bucket_sizes = [4, 12, 30]

    ema_data = pd.DataFrame()
    for subdomain, group in merged_df.groupby('Subdomain'):
        subdomain_ema = pd.DataFrame({'Time': group['Time'], 'Subdomain': subdomain})
        for bucket_size in bucket_sizes:
            ema_column_name = f'EMA_{bucket_size}_days'
            ema_values = group['Sentiment'].ewm(span=bucket_size, adjust=False).mean()
            subdomain_ema[ema_column_name] = (ema_values).astype(int)
        ema_data = pd.concat([ema_data, subdomain_ema])

    # Reset index
    ema_data.reset_index(drop=True, inplace=True)

    return ema_data

In [52]:
test = calculate_sliding_bucket_ema(sentiment_df)
test

C:\Users\sasre\AppData\Local\Temp\ipykernel_37484\827833874.py:23: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,Time,Subdomain,EMA_4_days,EMA_12_days,EMA_30_days
0,2024-03-09,Articles,50,50,50
1,2024-03-10,Articles,50,50,50
2,2024-03-11,Articles,50,50,50
3,2024-03-12,Articles,50,50,50
4,2024-03-13,Articles,50,50,50
...,...,...,...,...,...
197,2024-04-10,Real-Estate,34,36,36
198,2024-04-11,Real-Estate,34,35,36
199,2024-04-12,Real-Estate,30,34,35
200,2024-04-13,Real-Estate,18,28,33


In [54]:
from Feature_Engineering import generate_ema

In [57]:
categorical_ema, sliding_ema = generate_ema()

In [72]:
def create_gauge_chart(categorical_ema):
    # Define available Subdomains and EMA window sizes
    subdomains = categorical_ema['Subdomain'].unique()
    ema_window_sizes = ['EMA_4_days', 'EMA_12_days', 'EMA_30_days']
    
    # Define labels for EMA window sizes
    window_labels = {'EMA_4_days': 'Daily', 'EMA_12_days': 'Weekly', 'EMA_30_days': 'Monthly'}

    # Define initial Subdomain and EMA window size
    initial_subdomain = subdomains[0]
    initial_ema_window_size = ema_window_sizes[0]

    # Initial Subdomain and EMA window size
    selected_subdomain = initial_subdomain
    selected_ema_window_size = initial_ema_window_size

    # Filter data for selected Subdomain and EMA window size
    filtered_data = categorical_ema[(categorical_ema['Subdomain'] == selected_subdomain)]

    # Create gauge chart
    gauge_chart = go.Figure(go.Indicator(
        mode="gauge+number",
        value=filtered_data[selected_ema_window_size].iloc[0],
        domain={'x': [0, 1], 'y': [0, 1]},
        gauge={'axis': {'range': [0, 100]},
               'bar': {'color': "darkblue"},
               'steps': [
                   {'range': [0, 20], 'color': "red"},
                   {'range': [20, 40], 'color': "orange"},
                   {'range': [40, 60], 'color': "yellow"},
                   {'range': [60, 80], 'color': "lightgreen"},
                   {'range': [80, 100], 'color': "green"}]}))

    # Add label underneath
    gauge_chart.add_annotation(
        text=f"{selected_subdomain}",
        x=0.5,
        y=-0.3,
        showarrow=False,
        font=dict(size=16)
    )

    # Update layout
    gauge_chart.update_layout(
        title={'text': f"<b>{window_labels[selected_ema_window_size]} Fear and Greed Index</b>"},
        width=600,
        height=400,
        margin=dict(t=100)
    )

    # Show chart
    gauge_chart.show()


In [73]:
create_gauge_chart(categorical_ema)